In [1]:
%matplotlib qt5

In [2]:
# coding=utf-8

import numpy as np
from argparse import Namespace

from PIL import Image
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

camera_matrix = {'xc': 127.5, 'zc': 127.5, 'f': 128}
camera_matrix = Namespace(**camera_matrix)
def get_point_cloud_from_z(Y, camera_matrix, scale=1):
    x, z = np.meshgrid(np.arange(Y.shape[-1]),
                       np.arange(Y.shape[-2] - 1, -1, -1))
    for i in range(Y.ndim - 2):
        x = np.expand_dims(x, axis=0)
        z = np.expand_dims(z, axis=0)
    X = (x[::scale, ::scale] - camera_matrix.xc) * Y[::scale, ::scale] / camera_matrix.f
    Z = (z[::scale, ::scale] - camera_matrix.zc) * Y[::scale, ::scale] / camera_matrix.f
    XYZ = np.concatenate((X[..., np.newaxis], Y[::scale, ::scale][..., np.newaxis]
                          ,Z[..., np.newaxis]), axis=X.ndim)
    return XYZ

In [3]:
img_noise = "depth2.png"

In [4]:
image = Image.open(img_noise)
img = np.asarray(image)
img[img<1000] = 0
img[img>2000] = 0

In [5]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [6]:
from sklearn.cluster import DBSCAN

In [7]:
x, y = np.indices(img.shape)
idx = img > 0
data = np.stack([x[idx], y[idx], img[idx]]).T

In [8]:
data.shape

(113840, 3)

In [9]:
recon = np.zeros_like(img)
recon[data[:,0], data[:,1]] = data[:,2]
assert (recon == img).all()

In [10]:
cluter = DBSCAN(eps=5, min_samples=5).fit(data)
cluter.labels_

array([-1, -1, -1, ..., -1, -1, -1], dtype=int64)

In [11]:
from collections import Counter
ct = Counter(cluter.labels_)

In [12]:
keys = set(k for k, v in ct.most_common()[:10] if k != -1)
iidx = np.zeros_like(cluter.labels_, dtype=bool)
for k in keys:
    iidx = np.logical_or(iidx, cluter.labels_ == k)

In [13]:
new_data = data.copy()
new_data[:, 2][~iidx] = 0

recon = np.zeros_like(img)
recon[new_data[:,0], new_data[:,1]] = new_data[:,2]

In [14]:
XYZ = get_point_cloud_from_z(recon,camera_matrix,scale=1)
ax = plt.axes(projection='3d')
ax.plot(np.ndarray.flatten(XYZ[::,::,0]),np.ndarray.flatten(XYZ[::,::,1]),np.ndarray.flatten( XYZ[::,::,2]), 'b.', markersize=0.2)

plt.title('point cloud')
plt.show()

In [15]:
Image.fromarray(recon).save('filtered.png')